# Kobe Bryant Shot Selection ML

Este é um projeto de Engenharia de Machine Learning e tem o objetivo de utilizar a base de dados kobe-bryant-shot-selection disponível no
site Kaggle, URL: https://www.kaggle.com/c/kobe-bryant-shot-selection/data. Essa base de dados trás informações como circustâncias e localização, entre outras,
dos arremessos realizados pelo astro da NBA Kobe Bryant durante sua carreira. A intenção é determinar através dos algoritmos de machine learning
de foi convertida a cesta, variável alvo shot_made_flag.

# 1. Repositório e Template

Este projeto possue o seguinte repositório de dados URL: https://github.com/eriktavares/KobeBryantShotSelectionML. As estruturas de diretórios de arquivos foram baseadas no padrão Framework TDSP da Microsoft, e foi baixo o template pela URL https://github.com/Azure/Azure-TDSP-ProjectTemplate. Somente a pasta Simple_Data foi renomeada para Data, por conta da descrição que foi solicitado no enunciado da atividade (moodle). O arquivo de dados foi renomeado para 

In [1]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

C:\ProgramData\Anaconda3\envs\user01a\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: MLflow support for Python 3.6 is deprecated and will be dropped in an upcoming release. At that point, existing Python 3.6 workflows that use MLflow will continue to work without modification, but Python 3.6 users will no longer get access to the latest MLflow features and bugfixes. We recommend that you upgrade to Python 3.7 or newer.
  if __name__ == '__main__':


Carregamento dos Dados

# 2. Diagrama MLOps

# 3. Pepilines

# 4. Ferramentas

# Exploração dos Dados

In [2]:
df_kb = pd.read_csv('../Data/kobe_dataset.csv')
target_col = 'shot_made_flag'


#label_map = df_wine[['target', 'target_label']].drop_duplicates()
#drop_cols = ['target_label']
#df_wine.drop(drop_cols, axis=1, inplace=True)
#print(df_kb.shape)

#df_kb.head()
#df_kb.keys()

# Sklearn

# Pycaret

# MLFLOW

Para realizar o gerenciamento do ciclo de vida deste projeto de machine learning, será utilizado o MLFLOW. Conforme a descrição do site "O MLflow é uma plataforma de código aberto para gerenciar o ciclo de vida do ML, incluindo experimentação, reprodutibilidade, implantação e um registro de modelo central. Atualmente, o MLflow oferece quatro componentes: " https://mlflow.org/
- MLflow Tracking
- MLflow Projects
- MLflow Models
- Model Registry

Para reduzir a complexidade de codificação e simplificar a automação do processo de ML, será utilizado o Pycaret. 

PyCaret é uma biblioteca de aprendizado de máquina de código aberto e de baixo código em Python que automatiza fluxos de trabalho de aprendizado de máquina. É uma ferramenta de aprendizado de máquina e gerenciamento de modelos de ponta a ponta que acelera exponencialmente o ciclo de experimentos e torna você mais produtivo.
Comparado com outras bibliotecas de aprendizado de máquina de código aberto, o PyCaret é uma biblioteca alternativa de baixo código que pode ser usada para substituir centenas de linhas de código por apenas algumas linhas. Isso torna os experimentos exponencialmente rápidos e eficientes. https://pycaret.gitbook.io/docs/

Set up do MLFlow Server, executado no notebook MFLOWSetup

In [3]:
#!mlflow server --backend-store-uri sqlite:///mlruns.db --default-artifact-root ./artifacts --host 127.0.0.1

# 5. Artefatos

# Definição do Experimento 

Aqui está sendo definido o experimento para log dentro do MLFlow, os dados do experimento serão armazenados no banco mlruns.db,
e será utilizado o SQLite como banco de dados. O experimento foi definido como 'Kobe_Bryant_Shot_Experiment'

In [4]:
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Kobe_Bryant_Shot_Experiment'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

# 6. PreparacaoDados

A preparação dos dados é um passo importante no processo de Auto ML, nesta etapa será carregado o tratado os dados para os processos 

seguintes. Os dados são carregados do arquivo ../Data/kobe_dataset.csv que veio do site kaggle, o tamanho inicial foi registrado
com o nome Tamanho/Linhas - Base Entrada. 
A variável alvo 'shot_made_flag' está com dados faltantes, a quantidade de linhas foi registrado como Quantidade de shot_made_flag Faltante

O tamanho resultante da remoção dos dados faltantes foi registrado como Tamanho/Linhas - Base sem dados faltantes.
Posteriormente foi filtrada para somente os dados com arremeços de 2 pontos 2PT Field Goal e salvo em ../Data/Processed/data_filtered.parquet

Essa base foi separada em treino/teste 80% e 20% para operação. Registrados os tamanhos no MLFlow como Tamanho/Linhas - Base Treino/Teste e Tamanho/Linhas - Base Operação, respectivamente. Salvos em '../Data/Operalization/base_train_test.parquet' e
'../Data/Operalization/base_operation.parquet'

Os dados com arremeços de 3 pontos 3PT Field Goal, registrado o tamanho Tamanho/Linhas - Base Novidade
e armazenado em '../Data/Operalization/base_novelty.parquet'.


Name	Value
Quantidade de shot_made_flag Faltante	5000

Tamanho/Linhas - Base Entrada	30697

Tamanho/Linhas - Base Novidade	5412

Tamanho/Linhas - Base Operação	4057

Tamanho/Linhas - Base Treino/Teste	16228

Tamanho/Linhas - Base sem dados faltantes	25697


Essa separação dos dados treino/teste foi feita utilizando shuffle=True para que seja feito de forma aleatória, e o parâmetro stratify array como default, garantido que seja aleatória e estratificada. Aleatória que os dados serão misturados, e o estratificado garante a proporcionalidade das amostras. Essa técnica evita que os dados sejam divididos de forma a não expressão
a real exencia da informação. Por exemplo, se todos os dados de cesta convertidos estivessem no inicio do dataset ou os erros no final, uma divisão mantendo essa ordenação, iria disponibilizar para o modelo, informação agrupopada com uma tendência predominando, diferente dos dados totais. Dessa forma a modelagem ficaria prejudidada, assim como a validação com os dados de teste. Assim, a aleatóriedade e a manutenção das proporcionalizade na divisão dos dados, garante que o modelo esta recebendo a informação coerênte a totalizadade dos dados.    

In [8]:
# COLOCAR RUN DE LEITURA DE DADOS
# PARAMETROS: top_features,
# METRICS: SHAPE de cada base de dados
# ARTIFACTS: nenhum


top_features = ['lat','lon', 'minutes_remaining' , 'period', 'playoffs', 'shot_distance']
target_col = 'shot_made_flag'
target_col_label = 'shot_made_label'

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados', nested=True):
    
    #Leitura de dados
    path_kb_data_input= '../Data/kobe_dataset.csv'
    df_kb_all = pd.read_csv(path_kb_data_input)    
    mlflow.log_metric("Tamanho/Linhas - Base Entrada", df_kb_all.shape[0])
    
    #Descrição Variável alvo
    mapa ={0 : 'Errou', 1 : 'Cesta'}
    df_kb_all['shot_made_label'] = pd.DataFrame(df_kb_all [target_col].map(mapa))
    df_kb_all[[target_col, target_col_label]]
    
    
    #Remoção de dados Faltantes na Shot_made_Flag
    mlflow.log_metric("Quantidade de {} Faltante".format(target_col), df_kb_all['shot_made_flag'].isnull().sum())
    df_kb = df_kb_all[df_kb_all['shot_made_flag'].notnull()].reset_index()
    mlflow.log_metric("Tamanho/Linhas - Base sem dados faltantes", df_kb.shape[0])
    
    
    #Seleção de Features
    df_kb_tf = df_kb [top_features + ['shot_type', target_col]].copy()
    mlflow.log_param("top_features", top_features)
    
    #Filtro 2PT Field Goal
    
    df_kb_2PT = df_kb_tf[df_kb_tf['shot_type'] == '2PT Field Goal'].copy().drop('shot_type', axis=1)
    df_kb_2PT.to_parquet('../Data/Processed/data_filtered.parquet')
    
    
    # Separação da base com 80%/20% test_size=0.2
    #stratifyarray-like, default=None If not None, data is split in a stratified fashion, using this as the class labels.
    #shuffle = True
    df_kb_tt, df_kb_operation, ytrain, ytest = model_selection.train_test_split(df_kb_2PT, 
                                                                            df_kb_2PT[target_col],
                                                                            test_size=0.2,
                                                                            shuffle=True)
    
    mlflow.log_param("Percentual Operação", '0.2')
    df_kb_tt[target_col]      = ytrain
    df_kb_operation[target_col] = ytest
    
    
    mlflow.log_metric("Tamanho/Linhas - Base Treino/Teste", df_kb_tt.shape[0])
    mlflow.log_metric("Tamanho/Linhas - Base Operação", df_kb_operation.shape[0])
    
    
    #Base  3PT Field Goal
    df_kb_novelty = df_kb[df_kb['shot_type'] == '3PT Field Goal'].copy().drop('shot_type', axis=1)
    mlflow.log_metric("Tamanho/Linhas - Base Novidade", df_kb_novelty.shape[0]) 
    
    #Envio datasets para "/Data/operalization/base_{train|test}.parquet
    df_kb_tt.to_parquet('../Data/Operalization/base_train_test.parquet')
    df_kb_operation.to_parquet('../Data/Operalization/base_operation.parquet')
    df_kb_novelty.to_parquet('../Data/Operalization/base_novelty.parquet')
    
#label_map = df_wine[['target', 'target_label']].drop_duplicates()
#drop_cols = ['target_label']
#df_wine.drop(drop_cols, axis=1, inplace=True)
#print(df_kb.shape)

#df_kb.head()
#df_kb.keys()
    
    
mlflow.end_run()

C:\ProgramData\Anaconda3\envs\user01a\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\user01a\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# 7.Treinamento

Essa função inicializa o experimento no PyCaret e cria o pipeline de transformação com base em todos os parâmetros passados ​​na função. A função de configuração deve ser chamada antes de executar qualquer outra função. São necessários dois parâmetros obrigatórios: data e destino. Todos os outros parâmetros são opcionais. https://pycaret.gitbook.io/docs/get-started/functions/initialize#setting-up-environment.

Neste caso os parâmetros obrigatórios são df_kb_tt (base de dados) e o nome da coluna da váriável alvo.

Os parâmetros para gerar os logs do experimento no MLFLOW.

    - log_experiment = True, 
    - experiment_name = experiment_name, 
    - log_plots = True


As metricas default do Pycaret são: 'Accuracy' 'AUC', 'Recall', 'Precision', 'F1', 'Kappa', 'MCC'. Porém será adicionado também
a Metrica Perda de Log

Perda de log, também conhecida como perda logística ou perda de entropia cruzada.

Esta é a função de perda usada na regressão logística (multinomial) e em suas extensões, como redes neurais, definida como a probabilidade logarítmica negativa de um modelo logístico que retorna probabilidades y_pred para seus dados de treinamento y_true. A perda de log é definida apenas para dois ou mais rótulos
Adicionando Metric Loss Log. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html

In [29]:
import pycaret.classification as pc
from sklearn.metrics import log_loss
#pc.remove_metric('logloss') 
pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)
pc.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001CF7FC13840>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<function roc_auc_score at 0x000001CF7FBFD1E0>,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000001CF7FC13...,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000001CF7FC1C...,make_scorer(matthews_corrcoef),pred,{},True,True,False
logloss,LogLoss,LogLoss,<function log_loss at 0x000001CF7FC232F0>,"make_scorer(log_loss, greater_is_better=False)",pred,{},False,True,True


# 7a - Regressão Logistica

In [39]:
#import pycaret.classification as pc
# COLOCAR RUN DE TREINAMENTO DE MODELOS
# PARAMETROS: fold_strategy, fold, model_name, registered_model_name, cross_validation
# METRICS: auto sklearn
# ARTIFACTS: plots
# add Log Loss metric in pycaret

registered_model_name = 'modelo_regressão_kb'
model_name = 'lr'
probability_threshold = 0.5
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10
with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento', nested=True):
    # train/test
    s = pc.setup(data = df_kb_tt, 
                 target = target_col,
                 train_size=0.7,
                 silent = True,
                 fold_strategy = 'stratifiedkfold',
                 fold = fold,
                 log_experiment = True, 
                 experiment_name = experiment_name, 
                 log_plots = True
                )
    pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)
    bestmodel = pc.create_model(model_name,
                                cross_validation = cross_validation, 
                                probability_threshold=probability_threshold)

    # Log do run, e nao do modelo respectivo
    classification_plots = [ 'auc','pr','confusion_matrix',
                          #'error', 'class_report', 
                        'threshold', 'f1', 'logloss',
                         'learning','vc','feature',
                       ]
    for plot_type in classification_plots:
        print('=> Aplicando plot ', plot_type)
        try:
            artifact = pc.plot_model(bestmodel, plot=plot_type, save=True, use_train_data=False)
            mlflow.log_artifact(artifact)
        except:
            print('=> Nao possivel plotar: ', plot_type )
            continue

    #pc.save_model(bestmodel, f'./{registered_model_name}') 
    # Carrega novamente o pipeline + bestmodel
    #model_pipe = pc.load_model(f'./{registered_model_name}')


mlflow.end_run()

INFO:logs:Saving 'Feature Importance.png'
INFO:logs:Visual Rendered Successfully
INFO:logs:plot_model() succesfully completed......................................


In [40]:
predict = bestmodel()

CustomProbabilityThresholdClassifier(C=1.0, class_weight=None,
                                     classifier=LogisticRegression(C=1.0,
                                                                   class_weight=None,
                                                                   dual=False,
                                                                   fit_intercept=True,
                                                                   intercept_scaling=1,
                                                                   l1_ratio=None,
                                                                   max_iter=1000,
                                                                   multi_class='auto',
                                                                   n_jobs=None,
                                                                   penalty='l2',
                                                                   random_state=7606,
                         

In [ ]:
# COLOCAR RUN APROVACAO DE MODELO
# PARAMETROS: min_precision
# METRICS: new_version, precision
# ARTIFACTS: None


with mlflow.start_run(experiment_id=experiment_id, run_name = 'Regressão Logistica', nested=True):
    pred_holdout = pc.predict_model(bestmodel)
    ll = sklearn.metrics.log_loss(pred_holdout[wine_target_col], pred_holdout['Label'])
    if pr > min_precision:
        print(f'=> Aceito o modelo com precisão {pr} (min: {min_precision})')
        pred_holdout.to_parquet('modelo_vinho_teste.parquet')
        # Assinatura do Modelo Inferida pelo MLFlow
        model_features = list(data_wine.drop(wine_target_col, axis=1).columns)
        inf_signature = infer_signature(data_wine[model_features], model_pipe.predict(data_wine))
        # Exemplo de entrada para o MLmodel
        input_example = {x: data_wine[x].values[:nexamples] for x in model_features}
        # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
        mlflow.sklearn.log_model(
            sk_model=model_pipe,
            artifact_path="sklearn-model",
            registered_model_name=registered_model_name,
            signature = inf_signature,
            input_example = input_example
        )
        # Criacao do cliente do servico MLFlow e atualizacao versao modelo
        client = MlflowClient()
        if model_version == -1:
            model_version = client.get_latest_versions(registered_model_name)[-1].version
        # Registrar o modelo como staging
        client.transition_model_version_stage(
            name=registered_model_name,
            version=model_version, # Verificar com usuario qual versao
            stage="Staging"
        )
    else:
        print(f'=> Rejeitado o modelo com precisão {pr} (min: {min_precision})')

    # LOG DE PARAMETROS DO MODELO
    mlflow.log_param("precisao_minima", min_precision)

    # LOG DE METRICAS GLOBAIS
    mlflow.log_metric("new_version", model_version)
    mlflow.log_metric("precisao", pr)

mlflow.end_run()

# 7C - Arvore de Decisão

In [36]:
import pycaret.classification as pc
# COLOCAR RUN DE TREINAMENTO DE MODELOS
# PARAMETROS: fold_strategy, fold, model_name, registered_model_name, cross_validation
# METRICS: auto sklearn
# ARTIFACTS: plots
# add Log Loss metric in pycaret




registered_model_name = 'modelo_regressão_kb'
model_name = 'dt'
probability_threshold = 0.5
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10
with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento', nested=True):
    # train/test
    s = pc.setup(data = df_kb_tt, 
                 target = target_col,
                 train_size=0.7,
                 silent = True,
                 fold_strategy = 'stratifiedkfold',
                 fold = fold,
                 log_experiment = True, 
                 experiment_name = experiment_name, 
                 log_plots = True
                )
    pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)
    bestmodel = pc.create_model(model_name,
                                cross_validation = cross_validation, 
                                probability_threshold=probability_threshold)

    # Log do run, e nao do modelo respectivo
    #classification_plots = [ 'f1','logloss']
   # for plot_type in classification_plots:
       # print('=> Aplicando plot ', plot_type)
       # try:
        #    artifact = pc.plot_model(bestmodel, plot=plot_type, save=True, use_train_data=False)
        #    mlflow.log_artifact(artifact)
        #except:
         #   print('=> Nao possivel plotar: ', plot_type )
         #   continue

    #pc.save_model(bestmodel, f'./{registered_model_name}') 
    # Carrega novamente o pipeline + bestmodel
    #model_pipe = pc.load_model(f'./{registered_model_name}')


mlflow.end_run()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.5475,0.5285,0.5996,0.5185,0.5561,0.0997,0.1008,15.6278
1,0.5185,0.4887,0.5791,0.4921,0.5321,0.0429,0.0435,16.6311
2,0.5572,0.5489,0.5885,0.5284,0.5568,0.1170,0.1177,15.2933
3,0.5308,0.5184,0.5493,0.5034,0.5254,0.0632,0.0635,16.2055
4,0.5511,0.5327,0.5885,0.5223,0.5534,0.1053,0.1061,15.5062
5,0.5519,0.5317,0.5959,0.5229,0.5570,0.1076,0.1086,15.4758
6,0.5423,0.5194,0.5922,0.5137,0.5502,0.0890,0.0899,15.8102
7,0.5563,0.5397,0.6101,0.5257,0.5648,0.1174,0.1188,15.3237
8,0.5563,0.5372,0.5951,0.5264,0.5587,0.1160,0.1169,15.3237


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:CustomProbabilityThresholdClassifier(ccp_alpha=0.0, class_weight=None,
                                     classifier=DecisionTreeClassifier(ccp_alpha=0.0,
                                                                       class_weight=None,
                                                                       criterion='gini',
                                                                       max_depth=None,
                                                                       max_features=None,
                                                                       max_leaf_nodes=None,
                                                                       min_impurity_decrease=0.0,
                                                                       min_impurity_split=None,
                                                                       min_samples_leaf=

# 7C. Escolha Livre

Uma forma de realização de uma escolha para um algoritmo seria utilizar o função compare_models do Pycaret. O sort define o parâmetro de ordenação, nesse caso foi utilizado o Log Loss.

In [41]:
model = pc.compare_models(n_select = 1, sort='logloss')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
ada,Ada Boost Classifier,0.5943,0.5981,0.3686,0.6295,0.4642,0.1726,0.1889,14.0113,0.1000
gbc,Gradient Boosting Classifier,0.5895,0.6019,0.3896,0.6104,0.4751,0.1646,0.1759,14.1786,0.2420
ridge,Ridge Classifier,0.5811,0.0000,0.4933,0.5719,0.5295,0.1557,0.1572,14.4676,0.0100
lda,Linear Discriminant Analysis,0.5811,0.6008,0.4937,0.5718,0.5297,0.1558,0.1572,14.4676,0.0280
lr,Logistic Regression,0.5802,0.5983,0.4898,0.5711,0.5272,0.1536,0.1552,14.5010,0.5240
lightgbm,Light Gradient Boosting Machine,0.5781,0.5945,0.4826,0.5692,0.5221,0.1492,0.1508,14.5709,0.0510
rf,Random Forest Classifier,0.5618,0.5713,0.5345,0.5422,0.5382,0.1213,0.1214,15.1365,0.3820
et,Extra Trees Classifier,0.5546,0.5600,0.5544,0.5327,0.5432,0.1090,0.1092,15.3828,0.4050
nb,Naive Bayes,0.5541,0.5836,0.6301,0.5316,0.5721,0.1139,0.1174,15.4012,0.0140
knn,K Neighbors Classifier,0.5488,0.5631,0.5047,0.5293,0.5166,0.0941,0.0942,15.5835,0.0950


INFO:logs:create_model_container: 15
INFO:logs:master_model_container: 15
INFO:logs:display_container: 3
INFO:logs:AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=7606)
INFO:logs:compare_models() succesfully completed......................................


In [42]:
import pycaret.classification as pc
# COLOCAR RUN DE TREINAMENTO DE MODELOS
# PARAMETROS: fold_strategy, fold, model_name, registered_model_name, cross_validation
# METRICS: auto sklearn
# ARTIFACTS: plots
# add Log Loss metric in pycaret




registered_model_name = 'modelo_regressão_kb'
model_name = 'ada'
probability_threshold = 0.5
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10
with mlflow.start_run(experiment_id=experiment_id, run_name = 'Treinamento', nested=True):
    # train/test
    s = pc.setup(data = df_kb_tt, 
                 target = target_col,
                 train_size=0.7,
                 silent = True,
                 fold_strategy = 'stratifiedkfold',
                 fold = fold,
                 log_experiment = True, 
                 experiment_name = experiment_name, 
                 log_plots = True
                )
    pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)
    bestmodel = pc.create_model(model_name,
                                cross_validation = cross_validation, 
                                probability_threshold=probability_threshold)

    # Log do run, e nao do modelo respectivo
    #classification_plots = [ 'f1','logloss']
   # for plot_type in classification_plots:
       # print('=> Aplicando plot ', plot_type)
       # try:
        #    artifact = pc.plot_model(bestmodel, plot=plot_type, save=True, use_train_data=False)
        #    mlflow.log_artifact(artifact)
        #except:
         #   print('=> Nao possivel plotar: ', plot_type )
         #   continue

    #pc.save_model(bestmodel, f'./{registered_model_name}') 
    # Carrega novamente o pipeline + bestmodel
    #model_pipe = pc.load_model(f'./{registered_model_name}')


mlflow.end_run()

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.5933,0.6021,0.3457,0.6339,0.4474,0.1676,0.1870,14.0467
1,0.6100,0.6185,0.4030,0.6450,0.4960,0.2048,0.2199,13.4690
2,0.6074,0.5922,0.4159,0.6338,0.5022,0.2005,0.2127,13.5602
3,0.5819,0.5806,0.3346,0.6115,0.4325,0.1443,0.1608,14.4419
4,0.5607,0.5694,0.3309,0.5665,0.4177,0.1026,0.1121,15.1716
5,0.5827,0.5748,0.3401,0.6113,0.4371,0.1464,0.1624,14.4115
6,0.5871,0.5986,0.3826,0.6053,0.4689,0.1584,0.1696,14.2595
7,0.5933,0.5956,0.3487,0.6342,0.4500,0.1685,0.1876,14.0467
8,0.5871,0.5772,0.3506,0.6189,0.4476,0.1566,0.1727,14.2595


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:CustomProbabilityThresholdClassifier(algorithm='SAMME.R', base_estimator=None,
                                     classifier=AdaBoostClassifier(algorithm='SAMME.R',
                                                                   base_estimator=None,
                                                                   learning_rate=1.0,
                                                                   n_estimators=50,
                                                                   random_state=4906),
                                     learning_rate=1.0, n_estimators=50,
                                     probability_threshold=0.5,
                                     random_state=4906)
INFO:logs:create_model() succesfully completed......................................


# 8 Registro do Modelo

In [68]:
from mlflow.tracking import MlflowClient
import mlflow
#mlflow.set_registry_uri("sqlite:///mlruns.db")
#from mlflow.models.signature import infer_signature
from sklearn import tree, preprocessing, metrics, model_selection
#from mlflow.models.signature import ModelSignature

model_version = -1 
registered_model_name = 'Modelo Kobe Bryant'

with mlflow.start_run(experiment_id=experiment_id, run_name = 'RegistroModelo', nested=True):

    # Test set
    #pred_holdout = pc.predict_model(bestmodel)
    #pr = metrics.precision_score(pred_holdout[target_col], pred_holdout['Label'])
    #if pr > min_precision:
       # print(f'=> Aceito o modelo com precisão {pr} (min: {min_precision})')
        # Pycaret exporta junto o pipeline de preprocessamento
    pc.save_model(bestmodel, f'./{registered_model_name}') 
        # Carrega novamente o pipeline + bestmodel
    model_pipe = pc.load_model(f'./{registered_model_name}')
        # Assinatura do Modelo Inferida pelo MLFlow
    model_features = list(df_kb_tt.drop(target_col, axis=1).columns)
        #inf_signature = infer_signature(DataBin[model_features], model_pipe.predict(DataBin))
        # Exemplo de entrada para o MLmodel
        #input_example = {x: DataBin[x].values[:nexamples] for x in model_features}
        # Log do pipeline de modelagem do sklearn e registrar como uma nova versao
    mlflow.sklearn.log_model(
        sk_model=model_pipe,
        artifact_path="sklearn-model",
        registered_model_name=registered_model_name,
            #signature = inf_signature,
            #input_example = input_example
        )
        # Criacao do cliente do servico MLFlow e atualizacao versao modelo
    client = MlflowClient()
    if model_version == -1:
        model_version = client.get_latest_versions(registered_model_name)[-1].version
        # Registrar o modelo como staging
    client.transition_model_version_stage(
        name=registered_model_name,
        version=model_version, # Verificar com usuario qual versao
        stage="Staging"
    )

#else:
    #print(f'=> Rejeitado o modelo com precisão {pr} (min: {min_precision})')

mlflow.end_run()


INFO:logs:Initializing save_model()
INFO:logs:save_model(kwargs={}, verbose=True, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='shot_made_flag',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                nume...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
      

Transformation Pipeline and Model Successfully Saved


INFO:logs:Initializing load_model()
INFO:logs:load_model(verbose=True, authentication=None, platform=None, model_name=./Modelo Kobe Bryant)


Transformation Pipeline and Model Successfully Loaded


Registered model 'Modelo Kobe Bryant' already exists. Creating a new version of this model...
2022/04/16 08:59:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Modelo Kobe Bryant, version 3
Created version '3' of model 'Modelo Kobe Bryant'.


Ativando o serviço Server para o modelo Modelo Kobe Bryant em Staging, execução em outro notebook

In [69]:
#import os
#os.environ['MLFLOW_TRACKING_URI'] = 'sqlite:///mlruns.db'

#!mlflow models serve -m "models:/modelo_cancer/Staging" --no-conda -p 5001

# 8.1 Revalidação

Para a revalidação será feito utilizando os dados com 3PT Field Goal, que são diferentes porque o acerremeço de 3 pontos é mais
distânte do de 2 pontos. Dessa forma representa um novo conjunto de dados com caracteristicas, digamos que não esperadas pelo modelo,
que foi treinado com dados de arremeços de 2pts.

Abaixo esta uma função para calculo das principais metricas e retorno em formato dicionário.

In [96]:
def eval_metrics(actual, pred):
    return ({'Prec.': metrics.precision_score(actual, pred), 
             'Recall':metrics.recall_score(actual, pred),
             'F1':metrics.f1_score(actual, pred),
             'LogLoss':metrics.log_loss(actual, pred),
             'AUC':metrics.roc_auc_score(actual, pred),
             'Accuracy':metrics.accuracy_score(actual, pred),
             'Kappa':metrics.cohen_kappa_score(actual, pred),
             'MCC':metrics.matthews_corrcoef(actual, pred)})
            


O Serviço vai enviar uma request http para o serviço da API que realiza a predição e retorna os valores preditos em um JSON que é 
convertido para DataFrame e então são calculadas as metricas. Tudas as metricas são então salvas como log metric no MLFLow

In [94]:
import pandas as pd
import requests
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc


#Configuração do request
host = 'localhost'
port = '5001'
url = f'http://{host}:{port}/invocations'
headers = {'Content-Type': 'application/json',}

with mlflow.start_run(experiment_id=experiment_id, run_name = 'RevalidaçãoModelo', nested=True):
    #Dados para revalidação
    df_kb_op=pd.read_parquet('../Data/Operalization/base_operation.parquet')
    http_data = df_kb_op.drop(target_col,axis=1).to_json(orient='split')
    r = requests.post(url=url, headers=headers, data=http_data)
    df_kb_op.loc[:, 'operation_label'] = pd.read_json(r.text).values[:,0]

    #ll = log_loss(df_kb_op[target_col], df_kb_op['operation_label'])
    #f1 = f1_score(df_kb_op[target_col], df_kb_op['operation_label'])
    #acc= accuracy_score(df_kb_op[target_col], df_kb_op['operation_label'])
    #auc=auc(df_kb_op[target_col], df_kb_op['operation_label'])
    result= eval_metrics(df_kb_op[target_col], df_kb_op['operation_label'])
    result_title=''
    result_value=''
    for metric in result.keys():    
        mlflow.log_metric(metric, result[metric])
        print('{:<8}\t{:0.2f}'.format(metric, result[metric]))


mlflow.end_run()


Prec.   	0.63
Recall  	0.36
F1      	0.46
LogLoss 	14.01
AUC     	0.58
Accuracy	0.59
Kappa   	0.17
MCC     	0.19


Comparação

# 8.a Aderência com Novo Conjunto de Dados

In [137]:
df_ex = mlflow.search_runs([experiment_id], order_by=["metrics.m DESC"])
df_ex_fh = df_ex[df_ex['status'] == 'FINISHED'].copy()
df_ex_fh_rv = df_ex_fh[df_ex['tags.mlflow.runName'] == 'RevalidaçãoModelo'].copy()
df_ex_fh_rg = df_ex_fh[df_ex['tags.mlflow.runName'] == 'Ada Boost Classifier'].copy()
metrics_select = ['tags.mlflow.runName','metrics.LogLoss', 'metrics.F1','metrics.Accuracy', 'metrics.Prec.', 'metrics.Recall']
df_ex_fh_rv_fl=df_ex_fh_rv[metrics_select].copy()
df_ex_fh_rg_fl=df_ex_fh_rg[metrics_select].copy()
#print(df_ex_fh_rv_fl.keys())
df_rs=pd.concat([pd.DataFrame(df_ex_fh_rv_fl.iloc[:1]), df_ex_fh_rg_fl.iloc[:1]], axis=0)
df_rs       

,tags.mlflow.runName,metrics.LogLoss,metrics.F1,metrics.Accuracy,metrics.Prec.,metrics.Recall
0,RevalidaçãoModelo,14.013104,0.461035,0.594281,0.625778,0.364956
15,Ada Boost Classifier,14.199900,0.454700,0.588900,0.617200,0.360500


Na comparação entre os resultados obtidos no experimênto de Ada Boost Classifier que foi o melhor modelo escolhido pelo Pycaret
e o RelalidaçãoModelo que foi realizado com os dados de arremeços de 3 pontos. As metricas ficaram muito próximas, significa que 
não houve perca de performace.

# 8b Monitoramento do Modelo

# 8c Estrategias Reativa e Preditiva

# 9 Streamlit